# iOSとAndroidにおけるモバイルアプリのデータ分析

## ⚠ 免責事項

このプロジェクトは学習目的で作成されたものであり、  
企業内での業務や職歴を示すものではありません。

## プロジェクト概要

当社のモバイルアプリにおける収益構造は、アプリ内に表示される広告の閲覧数に大きく依存している。したがって、より多くのユーザーを当社のアプリに流入させることが、事業拡大における主要KPIのひとつである。

本プロジェクトでは、Kaggleから提供された[Apple Store](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps/data)と[Google Play Store](https://www.kaggle.com/datasets/lava18/google-play-store-apps)のパブリックデータセットを用いて、どのようなアプリがユーザーの注目を集めているかを分析する。その結果得られたインサイトをもとに、プロダクトマーケティングチームと連携し、データドリブンな意思決定の促進を図る。

## データの読み込み

まずは今回使用するApple StoreとGoogle Play StoreのCSVファイルを読み込む。データ分析時にカラムヘッダーが混在するのを防ぐため、ヘッダーとデータの中身を分けて設定しておく。

In [ ]:
# ファイルからデータを読み込み、リスト形式に変換
from csv import reader

opened_file_apple = open ('AppleStore.csv')
read_file_apple = reader(opened_file_apple)
apple_store_dataset = list(read_file_apple)
apple_header = apple_store_dataset[0]
apple_store_dataset = apple_store_dataset[1:]

opened_file_google = open('googleplaystore.csv')
read_file_google = reader(opened_file_google)
google_store_dataset = list(read_file_google)
google_header = google_store_dataset[0]
google_store_dataset = google_store_dataset[1:]

データセットの分析および欠陥検知の土台として、汎用的な関数を作成する。
この関数を用いて今回扱っていくデータの概要をつかむ。

In [ ]:
# 指定範囲の行データを表示し、必要に応じてデータセット及びスライスの行数・列数も表示する関数
def explore_data(dataset, start, end, dataset_R_and_C_number = False, slice_R_and_C_number = False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') #各rowの出力毎に改行を加える
        
    if dataset_R_and_C_number:
        print('Number of rows　in the dataset:', len(dataset))
        print('Number of columns in the dataset:', len(dataset[0]))
        
    if slice_R_and_C_number:
        print('Number of rows the slice:', len(dataset[start:end]))
        print('Number of columns in the slice:', len(dataset[0]))
        
print(apple_header)
print('\n')
print(explore_data(apple_store_dataset, 0, 3, True, True)) # スライスの行・列も同時に確認することでデータに欠損がないかを確認する

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows　in the dataset: 7197
Number of columns in the dataset: 16
Number of rows the slice: 3
Number of columns in the slice: 16
None


グーグルストアについても同様に調べる。

In [ ]:
# ヘッダーを表示させることで各データ要素の視認性を上げる
print(google_header)
print('\n')
print(explore_data(google_store_dataset, 0, 3, True, True))

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows　in the dataset: 10841
Number of columns in the dataset: 13
Number of rows the slice: 3
Number of columns in the slice: 13
None


## 異常データの削除

対象のデータセットを提供しているkaggleページのディスカッションにて、グーグルストアのデータセットの10472行目に欠陥があるという指摘があったため、ひとまず確認する。
比較を簡単にするために、以下の3点を可視化する。
- ヘッダー
- 欠陥のない行のサンプル
- 欠陥の疑いがある行

In [4]:
print(google_header)
print('\n')
print(google_store_dataset[0])
print('\n')
print(google_store_dataset[10472])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


3つ目の要素である'rating'が本来5段階評価のはずが19となっており、異常が発生していることがわかる。また、9つ目の要素も空白になっており、おそらく、この欠損によりほかのバリューの位置関係もずれていることが考えられる。

以上のことから、データの一貫性を担保するためにこの行は削除する。
念のため、そのデータが削除されているかの確認も含める。

In [ ]:
# KaggleのDiscussionで指摘された異常行（10472行目）を削除
print(len(google_store_dataset))
del google_store_dataset[10472]
print(len(google_store_dataset))

10841
10840


削除前のデータセットの行数と削除後の行数とで差が1なので、正常に削除が処理された確認が取れた。

## 重複するデータの除去

グーグルストアのデータには、重複しているデータも存在しているということもkeggleのディスカッションページで指摘されている。その一例として、インスタグラムに関するデータの重複が挙げられているので、まずはそれから確認をしてみる。

In [ ]:
# アプリ名がInstagramに対応するものを表示
for app in google_store_dataset:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


Instagramのデータの重複が確認できた。

このとき、4つ目の要素であるReviews（レビュー件数）のみ、各リスト異なる値をとっていることがわかる。

すべてのカテゴリの取得日は共通しているが、reviewsのみ差が生じていることから、考えられる
のは取得時間に違いがあるということである。従って、reviewsが多いほど、最近取得されたデータであると推測できる。

Instagramの場合は、4つある重複データのうち、reviewsの値が一番大きい3つ目のリストであるため、こちらを分析対象のデータに絞って分析業務を続行する。

同様に、重複データが存在するアプリに関しても、reviewsの値が最大のものをこれから分析の対象にできるよう、データの整理を行っていく。

---
これから、以下のことを行う。
- 重複データのあるアプリとそうでないアプリの数を明らかにして、全体像を把握する
- 辞書を用いて、各アプリにおいて、レビュー件数が最大のものが含まれているリストのみを抽出する
- 抽出したリストの中から、重複しているものを排除する

In [ ]:
# アプリ名の重複をチェックするためのリストを初期化
duplicate_apps = []
unique_apps = [] # 空リストは毎回初期化させるわけにはいかないため、forループの外側に配置する

for app in google_store_dataset:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('Number of unique apps:', len(unique_apps))
print('Number of duplicate apps:', len(duplicate_apps))
print('Double check number of unique apps:', len(google_store_dataset)-len(unique_apps)) # 計算が本当にあっているかを確認する
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:10])

Number of unique apps: 9659
Number of duplicate apps: 1181
Double check number of unique apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


In [ ]:
# 各アプリの最大レビュー数を記録（重複アプリの中で最も人気のあるものを抽出するため）
reviews_max = {}

for app in google_store_dataset:
    name = app[0]
    No_reviews = int(app[3])
    if name in reviews_max and reviews_max[name] < No_reviews:
        reviews_max[name] = No_reviews
    elif name not in reviews_max:
        reviews_max[name] = No_reviews

In [ ]:
google_clean = []
already_added = []

for app in google_store_dataset:
    name = app[0]
    No_reviews = int(app[3])
    
   # 最大レビュー数の行だけを追加（重複排除）、かつ二重追加を防ぐ
    if reviews_max[name] == No_reviews and name not in already_added:
        google_clean.append(app)
        already_added.append(name)

In [10]:
explore_data(google_clean, 0, 3, True, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows　in the dataset: 9659
Number of columns in the dataset: 13
Number of rows the slice: 3
Number of columns in the slice: 13


ここで、`google_clean`のリスト内にある行数(アプリの数)と、In[7]のコードセルで調べた`unique_apps`の数が9659で一致していることから、データの抽出が問題なく成功していることがわかる。

`apple_store_dataset`に関しては、すでに[kaggleのディスカッションページ](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps/discussion) の情報から重複データが存在しないことが予想されるが、一応念のために確認してみる。

In [11]:
apple_duplicate_apps = []
apple_unique_apps = []

for apps in apple_store_dataset:
    apps_id = app[0]
    if apps_id in unique_apps:
        duplicate_apps.append(apps_id)
    else:
        unique_apps.append(apps_id)

print('Number of unique apps:', len(apple_unique_apps))
print('Number of duplicate apps:', len(apple_duplicate_apps))
print('Double check number of unique apps:', len(apple_store_dataset)-len(apple_unique_apps)) # 計算が本当にあっているかを確認する
print('\n')
print('Examples of duplicate apps:', apple_duplicate_apps[:10])

Number of unique apps: 0
Number of duplicate apps: 0
Double check number of unique apps: 7197


Examples of duplicate apps: []


実際に、アップルストアのデータセットには重複アプリがないことが確認できた。

## 有料アプリの除去

弊社の製品は英語話者向けのコンテンツであるため、データセットの中から英語のアプリ以外を取り除いていく作業に取り掛かる。

英語で使われるアルファベットや記号などはASCII(American Standard Code for Infromation Interchange)というシステム基準が使われており、識別番号がそれぞれに振り当てられているので、それを利用して英語以外のアプリを除去する。

この際、ASCIIを識別できる組み込み関数の`ord()`を使って処理する。

また、ASCIIには絵文字や特別な記号などは含まれていないので、それらの文字が含まれるアプリをなるべく取り逃さないためも、「ASCII基準の文字・記号がが3語より多く含まれているアプリのみを排除する関数」を定義する。

In [ ]:
# 英語アプリだとみなせる条件を満たすアプリ以外を排除する関数
def is_english(string):
    non_ascii = 0
    
    for characters in string:
        if ord(characters) > 127:
            non_ascii += 1
            
    if non_ascii > 3:
        return False
    else:
        return True

定義した`is_english()`関数を用いて、アップルストアとグーグルストアのデータセットから英語のみのアプリの抽出を試みる。

In [ ]:
# 指定条件を満たす英語アプリを加えるための空リストを用意
apple_english = []
google_english = []

for apple_apps in apple_store_dataset:
    apple_name = apple_apps[1]
    if is_english(apple_name):
        apple_english.append(apple_apps)
        
        
for google_apps in google_clean:
    google_name = google_apps[0]
    if is_english(google_name):
        google_english.append(google_apps)
        
explore_data(apple_english, 0, 3, True, True)
print('\n')
explore_data(google_english, 0, 3, True, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows　in the dataset: 6183
Number of columns in the dataset: 16
Number of rows the slice: 3
Number of columns in the slice: 16


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5

英語アプリについて、アップルストアアプリでは7197個、グーグルストアのアプリでは10840個あることが分かった。


## 無料アプリの分離

次に、弊社の製品と同じ「無料」であるアプリを絞る。

In [ ]:
# 無料アプリを加えるための空リストを用意
free_english_apple = []
free_english_google = []

for apple_apps in apple_english:
    apple_price = apple_apps[4]
    if apple_price == '0.0':
        free_english_apple.append(apple_apps)
        
for google_apps in google_english:
    google_price = google_apps[7]
    if google_price == '0':
        free_english_google.append(google_apps)
        
print(len(free_english_apple))
print(len(free_english_google))

3222
8864


無料の英語アプリは、アップルストアでは3222個、グーグルストアでは8864個のサンプルが手元に入った。細かな条件設定を行っていけば、より正確な数字が求められると考えられるが、今プロジェクトの分析KPIの観点においては、十分なサンプル数であるためこれで一旦見切りをつける。

## 流通量が多いジャンル

イントロでも述べたように、私たちはどのようなアプリが需要が高いのかを調べることで、より多くのユーザーを獲得できるアプリ開発の実現を目指している。

アプリ開発のコスト削減のため、次のような施策を図る。

1. 最小限コストでAndroid版アプリを開発し、最初はグーグルストアで発信する
2. もしアプリが好評なら、さらに改善拡充を行う
3. もし6か月以上経ってもアプリが収益源になると判断されたら、IOS版も作成しアップルストアで発信する

このように、最終的にはアップルストアとグーグルストアの両方にアプリを掲載することが目標となるため、両方のプラットフォームで市場分析を行う必要がある。

---
まずは、アップルストアデータセットの`prime_genre`コラム、グーグルストアデータセットの`Genre`,`Category`コラムを参照しつつ辞書で頻度テーブルを作成する。

そのうえで、以下の2つの関数:

- 各ジャンルの頻度をパーセンテージで表すことができる関数
- 上記の頻度テーブルを降順で表すことができる関数

を定義し、分析をしやすくする。

補足：次のIn[15]のコードが個人的にチャレンジングだったため、セル内メモが多くあります。ご了承ください。

In [ ]:
# 指定データセットの列に関する頻度テーブルを作成する関数
def frequency_table_percentages(dataset, column_index):
    frequency_dict = {}
    total_count = 0

    for row in dataset: # このセクションでは、行の総数を求め、頻度テーブルを作成する
        total_count += 1 # 行数の文だけ+1される
        category = row[column_index] # apple/googleに使うめにindexは指定しない
        
        if category in frequency_dict:
            frequency_dict[category] += 1
        else:
            frequency_dict[category] = 1
    
    percentage_dict = {} # 上で作成した頻度テーブルを%で表すための処理を行う
    for category in frequency_dict:
        percentage = (frequency_dict[category] / total_count) * 100
        percentage_dict[category] = percentage # 作成した確率テーブルのキー経由でバリューを参照

    return percentage_dict # 作成した確率テーブルを絶対スコープに反映

 # 最初の関数で作成した確率テーブルを降順に示すための関数
def descend_table(dataset, column_index):
    percentage_table = frequency_table_percentages(dataset, column_index) # 確率テーブル
    percentage_list = [] # 降順で示したものを挿入するためのリスト

    for category in percentage_table:
        percentage_list.append((percentage_table[category], category)) # 割合とジャンル名をタプルにして用意した空リストに挿入する

    percentage_list_sorted = sorted(percentage_list, reverse=True)# sorted()は昇順する関数
                                                                  # デフォルトがreverse=False(昇順)だから、reverse=Trueにして降順にする
    for percentage, category in percentage_list_sorted:
        print(f"{category:<20} : {percentage:.2f}%") # f-string構文は文字列の中に変数を埋め込めるため、カスタマイズ性が高い。カテゴリ名と％を入れ替えて、見やすくする。
                                                # :.2fで小数点以下をどこまで含めるかを指定。これはf-stringとセットで使える書き方
                                                # 数値を最後に文字列として出力するのは、出力フォーマットを統一することで、人間にもコンピューターにも意味が明確に伝わりやすくなり、さらにレポート・表示・保存などの加工も柔軟に行えるようにするため。

In [ ]:
# prime_genreのインデックス番号を確認するために一旦表示
print(apple_header)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


作成した関数を用いて、アップルストアの`prime_genre`の結果を確認する。

In [17]:
descend_table(free_english_apple, -5)

Games                : 58.16%
Entertainment        : 7.88%
Photo & Video        : 4.97%
Education            : 3.66%
Social Networking    : 3.29%
Shopping             : 2.61%
Utilities            : 2.51%
Sports               : 2.14%
Music                : 2.05%
Health & Fitness     : 2.02%
Productivity         : 1.74%
Lifestyle            : 1.58%
News                 : 1.33%
Travel               : 1.24%
Finance              : 1.12%
Weather              : 0.87%
Food & Drink         : 0.81%
Reference            : 0.56%
Business             : 0.53%
Book                 : 0.43%
Navigation           : 0.19%
Medical              : 0.19%
Catalogs             : 0.12%


ゲーム及びエンタメ系ジャンルが過半数を占めていて圧倒的に流通していることがわかる。このことにより利用者層は若者がメインの可能性が浮上してくるが、これはインストール数やレビュー数ではなく、あくまでも「どのカテゴリが一番**発信**されているか」の指標にしかならないため、安易にジャンルの**人気度**と紐づけてはいけない。

続いてグーグルストアの`Genre`を確認する

In [18]:
descend_table(free_english_google, -4)

Tools                : 8.45%
Entertainment        : 6.07%
Education            : 5.35%
Business             : 4.59%
Productivity         : 3.89%
Lifestyle            : 3.89%
Finance              : 3.70%
Medical              : 3.53%
Sports               : 3.46%
Personalization      : 3.32%
Communication        : 3.24%
Action               : 3.10%
Health & Fitness     : 3.08%
Photography          : 2.94%
News & Magazines     : 2.80%
Social               : 2.66%
Travel & Local       : 2.32%
Shopping             : 2.25%
Books & Reference    : 2.14%
Simulation           : 2.04%
Dating               : 1.86%
Arcade               : 1.85%
Video Players & Editors : 1.77%
Casual               : 1.76%
Maps & Navigation    : 1.40%
Food & Drink         : 1.24%
Puzzle               : 1.13%
Racing               : 0.99%
Role Playing         : 0.94%
Libraries & Demo     : 0.94%
Auto & Vehicles      : 0.93%
Strategy             : 0.91%
House & Home         : 0.82%
Weather              : 0.80%
Events     

先ほどのアップルストアの検索結果と異なり、エンタメ系が特筆して流通しているわけではなく、ツールやビジネス関連の実務寄りのアプリとエンタメ系のアプリが拮抗している。一般的な市場ターゲットの年齢層が比較的高めで、学生よりも社会人が多い可能性がある。

最後にグーグルストアの`Category`を確認する。

In [19]:
descend_table(free_english_google, 1)

FAMILY               : 18.91%
GAME                 : 9.72%
TOOLS                : 8.46%
BUSINESS             : 4.59%
LIFESTYLE            : 3.90%
PRODUCTIVITY         : 3.89%
FINANCE              : 3.70%
MEDICAL              : 3.53%
SPORTS               : 3.40%
PERSONALIZATION      : 3.32%
COMMUNICATION        : 3.24%
HEALTH_AND_FITNESS   : 3.08%
PHOTOGRAPHY          : 2.94%
NEWS_AND_MAGAZINES   : 2.80%
SOCIAL               : 2.66%
TRAVEL_AND_LOCAL     : 2.34%
SHOPPING             : 2.25%
BOOKS_AND_REFERENCE  : 2.14%
DATING               : 1.86%
VIDEO_PLAYERS        : 1.79%
MAPS_AND_NAVIGATION  : 1.40%
FOOD_AND_DRINK       : 1.24%
EDUCATION            : 1.16%
ENTERTAINMENT        : 0.96%
LIBRARIES_AND_DEMO   : 0.94%
AUTO_AND_VEHICLES    : 0.93%
HOUSE_AND_HOME       : 0.82%
WEATHER              : 0.80%
EVENTS               : 0.71%
PARENTING            : 0.65%
ART_AND_DESIGN       : 0.64%
COMICS               : 0.62%
BEAUTY               : 0.60%


`Family`というカテゴリが一番多いことがわかる。この要素以外の部分に関しては、先ほど調べた`Genre`と同じく、ツール・ビジネス系のアプリとエンタメ系のアプリが上位域で拮抗している。

これらの結果から、無料の英語アプリに関して、アップルストアでは主にエンタメアプリが主に流通しており、一方でグーグルストアではエンタメアプリや実務系アプリにおけるカテゴリごとの格差があまり見られず、流通アプリに偏りが少ないということが分かった。

しかし、先ほども述べたように今調べたデータはジャンルごとのアプリ数を示すものであり、利用者数や売り上げ数には言及していないため、どのジャンルが人気までかはわからない。

なので、これからアプリの「人気度」に直接関連する要素の分析を始める。

## ジャンルごとの人気アプリ

各ストアにおける人気アプリを特定するため、インストール数を基準に調べていく。
グーグルストアに関しては、`Installs`というコラムがあるためそちらを参照できるが、アップルストアにはインストール数に直接言及するコラムは存在しないため、レビュー数のコラム`rating_count_tot`で代用する。

まずは、アップルストアからレビュー数`rating_count_tot`を基準に人気アプリを特定していく。出力結果がみやすいようにレビュー数を降順にして表示する。

In [ ]:
# 各ジャンルの出現比率（%）を辞書形式で取得
apple_genre = frequency_table_percentages(free_english_apple, -5)

# 上で取得した辞書形式のデータをあとで挿入してリスト形式にするために空リストを用意
apple_genre_list = []

# ジャンルを跨いで重複処理をしないよう、ジャンルごとに値を初期化する設定を施す
for genre in apple_genre:
    total = 0
    len_genre = 0
    # 各ジャンルごとに平均レビューを算出
    for app in free_english_apple:
        app_genre = app[-5]
        if app_genre == genre:
            No_reviews = int(app[5])
            total += No_reviews
            len_genre += 1
            
    avg_No_reviews = total / len_genre
    apple_genre_list.append((genre, avg_No_reviews)) #ジャンルとレビューの位置を入れ替えれば後のsorted()の中身がよりシンプルなるが、lambda構文を使ってみたかったため、あえてそのままにした

# 平均レビュー数を多い順（降順）に並び替え
descend_apple = sorted(apple_genre_list, key=lambda x: x[1], reverse=True)

for genre, avg_No_reviews in descend_apple:
    print(f"{genre:<20} : {avg_No_reviews:.0f}")

Navigation           : 86090
Reference            : 74942
Social Networking    : 71548
Music                : 57327
Weather              : 52280
Book                 : 39758
Food & Drink         : 33334
Finance              : 31468
Photo & Video        : 28442
Travel               : 28244
Shopping             : 26920
Health & Fitness     : 23298
Sports               : 23009
Games                : 22789
News                 : 21248
Productivity         : 21028
Utilities            : 18684
Lifestyle            : 16486
Entertainment        : 14030
Business             : 7491
Education            : 7004
Catalogs             : 4004
Medical              : 612


アップルストアでは、レビュー件数（インストール数の代用）に関しては`Nvigation`カテゴリが一番高いことが分かった。

また、In[17]の分析で明らかになった、流通度が最も高かったゲーム・エンタメカテゴリのレビュー件数がそこまで高くないことから、**需要があまり高くない上に競争率が高いジャンルである可能性が高い**と考えられる。

それでは、Navigationアプリの中でもどのようなアプリが人気なのかを具体的に調べていく。

In [ ]:
# 取り出したNavigationアプリだけを加えるための空リストを用意
navigation_apps = []

for app in free_english_apple:
    if app[-5] == 'Navigation':
        No_reviews = int(app[5])
        
        navigation_apps.append((app[1], No_reviews))
        
for name, reviews in navigation_apps:
    print(f"{name: <50} : {reviews:,}") # 見やすいように出力形式を整える

Waze - GPS Navigation, Maps & Real-time Traffic    : 345,046
Google Maps - Navigation & Transit                 : 154,911
Geocaching®                                        : 12,811
CoPilot GPS – Car Navigation & Offline Maps        : 3,582
ImmobilienScout24: Real Estate Search in Germany   : 187
Railway Route Search                               : 5


この結果により、Nagvigationカテゴリのレビュー数がとても多いのは、WazeやGoogle Mapsといったアプリの影響が大きいことが分かった。

このように、全体のレビュー平均数が多くても、超有名タイトルやブランドのアプリの影響が大きいと、必ずしもジャンル自体の人気度や需要を示す指標にならないことが分かる。このような現象は、音楽ならspotify、SNSならskypeやFacebookといった有名アプリの独占によって発生している可能性があると推測できる。

従って、これからの分析ではこのような超有名タイトルを排除し、よりフラットな目線でどのようなアプリが各ストアで人気なのかを調べていく必要がありそうだ。

とりあえず、先ほどの出力結果のランキング2位であるReferenceカテゴリを調べてみる。理由は、ほかの上位カテゴリと異なり、どのようなアプリが独占しているか想像がつきにくく謎が多いためだ。念のためにReferenceアプリは具体的などのようなアプリが人気なのかを調べる。

In [ ]:
# 取り出したReferenceアプリをだけを加えるための空リストを用意
reference_apps = []

for app in free_english_apple:
    if app[-5] == 'Reference':
        No_reviews = int(app[5])
        
        reference_apps.append((app[1], No_reviews))
        
descend_reference_apps = sorted(reference_apps, key=lambda x: x[1], reverse=True)

for name, reviews in descend_reference_apps:
    print(f"{name: <90} : {reviews:,}")
    print() # 出力結果がみやすいように改行を入れて表示

Bible                                                                                      : 985,920

Dictionary.com Dictionary & Thesaurus                                                      : 200,047

Dictionary.com Dictionary & Thesaurus for iPad                                             : 54,175

Google Translate                                                                           : 26,786

Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran                                         : 18,418

New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition                     : 17,588

Merriam-Webster Dictionary                                                                 : 16,849

Night Sky                                                                                  : 12,122

City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE)             : 8,535

LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools  

この結果より、Referecneカテゴリの主な人気アプリは聖書と辞書系アプリ、そしてゲームのMODだといえる。興味深いのは、聖書はさておき、これらのアプリに共通することは拡張的な機能としてその価値を発揮していることだ。

つまり、先ほどまでの`Navigation`や`Music`アプリ等とは異なり、ブランドやコンテンツではなくその機能自体に需要があるものだと考えられる。機能であれば、弊社のアプリに導入することができるかもしれないし、なにより辞書などの低予算でも実現可能性の高い機能なのが魅力的である。

この辞書機能はその柔軟性を鑑みてもポテンシャルがかなり高いといえる。辞書機能を拡張機能として弊社のアプリに導入することで、「メインのコンテンツ＋辞書機能」といった風に当社としても柔軟にサービスを展開できる可能性がある。

また、辞書などの機能面における拡張性は、ユーザーが辞書検索などをするためにアプリを離れる時間を減らすことに繋がり、ユーザーのアプリに対するエンゲージメントが維持される結果として広告収益の向上が見込めるため、検討の余地があるといえる。

読書アプリは先ほど挙げた辞書機能との相性は良いが、他の上位に食い込んでいる天気、飲食、ファイナンス系アプリに関しては、次のことが言える。

- **天気**: 
天気アプリは短時間の利用で済む特性があり、広告収益の最大化には繋がりにくいことが考えられる。また、天候データを取得するためのAPIが有料である可能性も危惧される。


- **飲食**: 
主にマックやスタバといった大手飲食店のアプリが独占していることが考えられる。実際に飲食サービスを手掛けている会社であれば開発価値があるものの、弊社はそのようなサービスを提供していないため領域外のジャンルであるといえる。


- **ファイナンス**: 
お金に関するアプリ開発には銀行・金融・資産運用などの専門知識が必要になってくるが、弊社としては専門家を雇うための出資はなるべく避けたいのが現実である。

----
アップルストアの人気アプリについてはここでひと段落つけて、次にグーグルストアの人気アプリのInstallsインストール数について調べる。

In [23]:
descend_table(free_english_google, 5)

1,000,000+           : 15.73%
100,000+             : 11.55%
10,000,000+          : 10.55%
10,000+              : 10.20%
1,000+               : 8.39%
100+                 : 6.92%
5,000,000+           : 6.83%
500,000+             : 5.56%
50,000+              : 4.77%
5,000+               : 4.51%
10+                  : 3.54%
500+                 : 3.25%
50,000,000+          : 2.30%
100,000,000+         : 2.13%
50+                  : 1.92%
5+                   : 0.79%
1+                   : 0.51%
500,000,000+         : 0.27%
1,000,000,000+       : 0.23%
0+                   : 0.05%
0                    : 0.01%


今回の分析では正確なインストール数を知る必要はなく、相対的にどのジャンルのアプリがインストール数が多いかを知ればいいので、インストール数の区分けはこのままで概算処理を続ける。

ただ、現段階では各インストール数の区分けに関する確率テーブルが出力されていて、どのジャンルのアプリがインストール数が多いのかわからないことが問題である。

なので、これからジャンルごとのインストール数とを示すための処理を行う。

In [ ]:
# In[20]と同じような処理を行う
google_category = frequency_table_percentages(free_english_google, 1)
category_installed = []

for genre in google_category:
    total = 0
    len_category = 0
    
    for app in free_english_google:
        if app[1] == genre:
            installs = app[5].replace(',', '').replace('+','') # 平均値を計算する上でノイズになる部分を除去する
            total += int(installs)
            len_category += 1
            
    avg_installs = total / len_category
    category_installed.append((genre, avg_installs))
    
category_installed_sorted = sorted(category_installed, key=lambda x: x[1], reverse=True)

for category, avg in category_installed_sorted:
    print(f"{category:<20} : {avg:,.0f}")

COMMUNICATION        : 38,456,119
VIDEO_PLAYERS        : 24,727,872
SOCIAL               : 23,253,652
PHOTOGRAPHY          : 17,840,110
PRODUCTIVITY         : 16,787,331
GAME                 : 15,588,016
TRAVEL_AND_LOCAL     : 13,984,078
ENTERTAINMENT        : 11,640,706
TOOLS                : 10,801,391
NEWS_AND_MAGAZINES   : 9,549,178
BOOKS_AND_REFERENCE  : 8,767,812
SHOPPING             : 7,036,877
PERSONALIZATION      : 5,201,483
WEATHER              : 5,074,486
HEALTH_AND_FITNESS   : 4,188,822
MAPS_AND_NAVIGATION  : 4,056,942
FAMILY               : 3,695,642
SPORTS               : 3,638,640
ART_AND_DESIGN       : 1,986,335
FOOD_AND_DRINK       : 1,924,898
EDUCATION            : 1,833,495
BUSINESS             : 1,712,290
LIFESTYLE            : 1,437,816
FINANCE              : 1,387,692
HOUSE_AND_HOME       : 1,331,541
DATING               : 854,029
COMICS               : 817,657
AUTO_AND_VEHICLES    : 647,318
LIBRARIES_AND_DEMO   : 638,504
PARENTING            : 542,604
BEAUTY     

この結果から、`COMUNICATION`カテゴリのアプリが一番インストール数が多いことがわかる。なので、これからその中身を調べる。

In [ ]:
communication_apps = []

# COMMUNICATION アプリの中でも、指定文字列（インストール数上位層）に該当するものだけを抽出
for app in free_english_google:
    if app[1] == 'COMMUNICATION' and app[5] in ('1,000,000,000+',
                                                '500,000,000+',
                                                '100,000,000+'):
        installs = int(app[5].replace(',', '').replace('+', '')) # 数字文字列を数値化し後に計算しやすいようにする
        
        communication_apps.append((app[0], installs))
        
descend_communication_apps = sorted(communication_apps, key=lambda x: x[1], reverse=True)

for name, installs in descend_communication_apps:
    print(f"{name: <50} : {installs:,}+")

WhatsApp Messenger                                 : 1,000,000,000+
Messenger – Text and Video Chat for Free           : 1,000,000,000+
Skype - free IM & video calls                      : 1,000,000,000+
Google Chrome: Fast & Secure                       : 1,000,000,000+
Gmail                                              : 1,000,000,000+
Hangouts                                           : 1,000,000,000+
Google Duo - High Quality Video Calls              : 500,000,000+
imo free video calls and chat                      : 500,000,000+
LINE: Free Calls & Messages                        : 500,000,000+
UC Browser - Fast Download Private & Secure        : 500,000,000+
Viber Messenger                                    : 500,000,000+
imo beta free calls and text                       : 100,000,000+
Android Messages                                   : 100,000,000+
Who                                                : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji         : 100,000,000

100,000,000インストール以上あるアプリは、超有名タイトルでカテゴリを独占しているアプリであるため、参考にならなそうだ。ただこのままでは印象論止まりなので、実際に全体の平均値（38,456,119個）と100,000,000インストール以上あるアプリを除いたアプリのインストール数の平均値を算出・比較して確認する。

In [ ]:
under_billion_app = []

for app in free_english_google:
    No_installs = int(app[5].replace(',', '').replace('+', ''))
    if app[1] == 'COMMUNICATION' and No_installs < 100_000_000: # 1億インストール未満のアプリを抽出し空リストに加える
        under_billion_app.append(No_installs)
        
avg_under_billion_installs = sum(under_billion_app) / len(under_billion_app)

print(f"average under 1 billion : {avg_under_billion_installs:.0f}")

average under 1 billion : 3603485


結果、全体の平均値は4千万弱、1億インストール以下のアプリの中での平均値は4百万弱であるため、全体の平均値は上位の独占アプリの影響を強く受けていることの確認が取れた。

このような現象は他の人気カテゴリでも発生していると予測できるが、全てのカテゴリを1から調べるのは合理性に欠ける。むしろ、すでにIn[22]で我々の開発プロダクトとの相性の良さが示唆されているReferenceカテゴリに焦点を当てるべきである。

よって、これからアップルストアにおけるReferenceカテゴリに対応するグーグルストアのREFERENCE_AND_BOOKSカテゴリ（In[24]を参照）について深堀する。

In [ ]:
# REFERENCE_AND_BOOKSアプリを加え、あとで並び替えるための空リストを用意
books_and_reference = []

for app in free_english_google:
    if app[1] == 'BOOKS_AND_REFERENCE':
        No_installs = int(app[5].replace(',', '').replace('+', ''))
        books_and_reference.append((app[0], No_installs))
        
descend_books_and_reference = sorted(books_and_reference, key=lambda x: x[1], reverse=True)

for name, installs in descend_books_and_reference:
    print(f"{name: <55} : {installs:,}+")

Google Play Books                                       : 1,000,000,000+
Bible                                                   : 100,000,000+
Amazon Kindle                                           : 100,000,000+
Wattpad 📖 Free Books                                    : 100,000,000+
Audiobooks from Audible                                 : 100,000,000+
Wikipedia                                               : 10,000,000+
Cool Reader                                             : 10,000,000+
FBReader: Favorite Book Reader                          : 10,000,000+
HTC Help                                                : 10,000,000+
Moon+ Reader                                            : 10,000,000+
Aldiko Book Reader                                      : 10,000,000+
Al-Quran (Free)                                         : 10,000,000+
Al Quran Indonesia                                      : 10,000,000+
Al'Quran Bahasa Indonesia                               : 10,000,000+
Quran for And

上位独占の有名タイトルをのぞけば、

- reader系の機能を持つアプリ
- 聖書の読書体験を向上させる機能をもつアプリ
- 辞書系のアプリ

がインストール数1千万以上の区域を占めており、需要が高いことの裏付けができた。

これらはIn[22]でも言及したように、コンテンツではなく機能として魅力を持つものであり、当社のアプリにも実装できる可能性がある。

## 結論

今回のプロジェクトでは、アップルストアとグーグルストアのデータセットから、当社のアプリ開発に役立ちそうな情報を分析した。

その結果、当社のアプリ開発KPIを達成するために、各ストアにおけるアプリの中で、そのジャンル毎のインストール数などを比較分析することで、以下の3点

- アップルストアとグーグルストアの両方で需要がある
- 低予算で実装可能である
- 広告収入型アプリとの相性が良い

の要素を持つアプリ開発のためのインサイトを得ることに努めた。結果、その一つの候補としてreference系アプリの「機能」面がそれに当てはまることが分かった。これはコンテンツ自体ではなく、機能的な魅力であるため、弊社のアプリに実装することが現実的に可能であり、検討の価値がある。In[22]でも言及したように、Reference系の機能とそれに相性の良いコンテンツを組み合わせれば、ユーザーがわざわざそれらの機能を使うためにアプリを離れる必要がなくなり、アプリのエンゲージ率の向上に伴い広告収益の向上も見込める。

また、避けるべき市場として、アップルストアにおけるエンタメ・ゲームアプリがあることが分かった。In[20]でも言及したように、需要の高さに対して供給が多すぎるカテゴリであるため、競争率が高く初期投資に対してROIが低くなるリスクが高いためである。

しかし、依然として「どのようなコンテンツのアプリにするか」「reference機能の中でもどのようなものがROIが最も高いか」などの分析の課題が残る。これらの課題は今回対象にした2つのデータセットだけでなく、ほかにも複数のデータセット、とくにSNSやレビューの定性分析も含めた複合的な分析を行うことで解消する必要があると考えられる。本分析は、こうした多面的なマーケティング判断の第一歩となる。